# <div align=center><b>PyTorch data loading</b></div>
Since many of the models trained on genomic sequence data nowadays are based on deep learning, we added functionality to SeqData to flexibly convert it to a PyTorch dataloader. This tutorial illustrates how to use this functionality to create...

In [1]:
import seqdata as sd
from pathlib import Path
sd.__version__

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as noteb

'0.1.2'

## SeqData to PyTorch dataloader

In [13]:
# Get file name
bw_fname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.bw'
bed_fname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.bed'
fasta_fname = Path(sd.__file__).resolve().parent.parent / 'tests' / 'data' / 'tangermeme.fa'
bw_fname, bed_fname, fasta_fname

(PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.bw'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.bed'),
 PosixPath('/cellar/users/aklie/projects/ML4GLand/SeqData/tests/data/tangermeme.fa'))

In [14]:
from seqdata import read_bigwig

In [15]:
sdata = read_bigwig(
    bigwigs=[bw_fname],  # bigwig files
    fasta=fasta_fname,  # reference genome
    seq_name="seq",  # name of resulting xarray variable containing sequences
    cov_name="cov",  # name of resulting xarray variable containing coverage
    bed=bed_fname,  # bed file with regions to extract
    samples=["tangermeme"],  # sample names
    out=bw_fname.with_suffix(".zarr"),
    fixed_length=True,  # whether all sequences are the same length
    batch_size=1000,  # number of sequences to load at once
    n_jobs=1,  # number of parallel jobs
    overwrite=True,  # overwrite the output file if it exists
)
sdata

100%|██████████| 5/5 [00:00<00:00, 14189.12it/s]


<xarray.Dataset>
Dimensions:     (_sequence: 5, cov_sample: 1, _length: 20)
Coordinates:
  * cov_sample  (cov_sample) object 'tangermeme'
Dimensions without coordinates: _sequence, _length
Data variables:
    chrom       (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
    chromEnd    (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    chromStart  (_sequence) int64 dask.array<chunksize=(5,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) float32 dask.array<chunksize=(5, 1, 20), meta=np.ndarray>
    seq         (_sequence, _length) |S1 dask.array<chunksize=(5, 20), meta=np.ndarray>
    strand      (_sequence) object dask.array<chunksize=(5,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    0
    sequence_dim:  _sequence

In [16]:
from seqdata import get_torch_dataloader

In [17]:
dataloader = get_torch_dataloader(
    sdata,
    sample_dims=["_sequence"],
    variables=["cov"],
    batch_size=10
)

In [18]:
next(iter(dataloader))

{'cov': tensor([[[1.3737, 1.4172, 1.2025, 0.8299, 1.7405, 1.4791, 0.4958, 0.1296,
           0.1112, 0.7721, 1.9588, 0.1598, 0.1070, 1.6203, 1.0502, 1.1368,
           0.4425, 0.8903, 0.3634, 1.3084]],
 
         [[1.7881, 1.9161, 0.3279, 0.4625, 0.4841, 0.2044, 0.2227, 0.3044,
           1.9460, 0.1615, 2.9371, 1.3581, 1.1392, 0.5296, 1.6804, 2.2249,
           1.0185, 0.3568, 1.8823, 0.6509]],
 
         [[0.8078, 1.5911, 0.8191, 0.0741, 1.4640, 0.4832, 1.0016, 0.6736,
           0.3631, 0.6796, 0.5143, 1.7786, 1.4153, 0.6761, 0.2795, 0.7449,
           0.5205, 0.6081, 0.0466, 0.7280]],
 
         [[0.3189, 0.6559, 0.5520, 0.2719, 0.8362, 0.8444, 0.5371, 0.2512,
           0.9365, 1.5987, 0.6665, 1.6923, 0.4150, 1.6298, 1.1279, 1.5984,
           0.2438, 0.8742, 0.9666, 0.6774]],
 
         [[0.6665, 1.6923, 0.4150, 1.6298, 1.1279, 1.5984, 0.2438, 0.8742,
           0.9666, 0.6774, 1.2272, 0.8674, 0.1969, 2.5658, 1.0753, 1.1059,
           0.1325, 0.2199, 0.0109, 1.6749]]])}

## Adding transforms

In [21]:
dataloader = get_torch_dataloader(
    sdata,
    sample_dims=["_sequence"],
    variables=["seq"],
    batch_size=10
)

Though we can successfully make a dataloader here, if we try to get a batch of sequences from it, it will throw an error.

In [22]:
next(iter(dataloader))

TypeError: can't convert np.ndarray of type numpy.bytes_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

This is because the type of data in the "seq" variable of the SeqData is 

In [23]:
import seqpro as sp


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



In [34]:
def one_hot_encode(batch):
    batch["seq"] = sp.ohe(batch["seq"], alphabet=sp.DNA)
    return batch

In [35]:
dataloader = get_torch_dataloader(
    sdata,
    sample_dims=["_sequence"],
    variables=["seq"],
    transform=one_hot_encode,
    batch_size=10
)

In [36]:
next(iter(dataloader))

{'seq': tensor([[[0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          [1., 0., 0., 0.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.]],
 
         [[0., 0., 0., 1.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          [0., 1., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 1., 0., 0.],
          [1., 0., 0., 0.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 1., 0., 0.],
          

## Combining dataloaders

In [ ]:
!pip install lightning

In [51]:
dataloader1 = get_torch_dataloader(
    sdata,
    sample_dims=["_sequence"],
    variables=["seq"],
    transform=one_hot_encode,
    batch_size=5
)

In [52]:
dataloader2 = get_torch_dataloader(
    sdata,
    sample_dims=["_sequence"],
    variables=["seq"],
    transform=one_hot_encode,
    batch_size=1
)

In [53]:
from lightning.pytorch.utilities import CombinedLoader

In [54]:
combined_loader = CombinedLoader({"dataset1":dataloader1, "dataset2": dataloader2}, mode='max_size_cycle')

In [62]:
for batch, batch_idx, dataloader_idx in combined_loader:
    print(f"{batch_idx=}, {dataloader_idx=}")

batch_idx=0, dataloader_idx=0
batch_idx=1, dataloader_idx=0
batch_idx=2, dataloader_idx=0
batch_idx=3, dataloader_idx=0
batch_idx=4, dataloader_idx=0


Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x155551176610>>
Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3952, in atexit_operations
    self._atexit_once()
  File "/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3935, in _atexit_once
    self.history_manager.end_session()
  File "/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/IPython/core/history.py", line 592, in end_session
    self.writeout_cache()
  File "/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/decorator.py", line 232, in fun
    return caller(func, *(extras + args), **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/cellar/users/aklie/opt/miniconda3/envs/eug

In [60]:
batch['dataset1']['seq'].shape, batch['dataset2']['seq'].shape

(torch.Size([5, 20, 4]), torch.Size([1, 20, 4]))

# DONE!

---